# Recitation 3

## Problem Description
A set of radiating catheters are used to kill cancerous cells in a tumor while preserving healthy cells. We wish to select a set of catheter locations and active times so that cancer cells release a lethal dose and healthy cells receive a nonlethal dose. The problem is described in more detail in Lecture 6 and Recitation 3 slides. We will implement the case where the number of catheters are limited and no neighboring catheters can be used at the same time.

In [1]:
##### Getting Started

# When starting to learn any programming language or tool, it's important to familiarize
# yourself with existing resources to help you learn.
# The official Julia documentation: https://docs.julialang.org/en/v1/ (or google "Julia docs")
# It consists of "The Manual" and documentation for the two modules automatically loaded
# whenever julia starts: Base and Standard Library.

# Julia code is packaged into "modules". This command loads the JuMP and Gurobi modules. 
# More info on Modules in general: https://docs.julialang.org/en/v1/manual/modules/
# Documentation for JuMP.jl: https://jump.dev/JuMP.jl/dev/ (or google JuMP.jl and click documentation in github ReadMe file)
# Documentation for Gurobi.jl: https://github.com/jump-dev/Gurobi.jl
using JuMP, Gurobi

In [2]:
typeof(Gurobi) # A function in the Core module. Documented: https://docs.julialang.org/en/v1/base/base/#Core.typeof

Module

In [3]:
names(Gurobi) # A function in the Base module that prints all "names" exported by a module. Documented: https://docs.julialang.org/en/v1/base/base/#Base.names

919-element Vector{Symbol}:
 :GRBBSolve
 :GRBBinvColj
 :GRBBinvRowi
 :GRBBinvi
 :GRBBinvj
 :GRBFSolve
 :GRBXaddconstrs
 :GRBXaddrangeconstrs
 :GRBXaddvars
 :GRBXchgcoeffs
 :GRBXgetconstrs
 :GRBXgetvars
 :GRBXloadmodel
 ⋮
 :presolve_model
 :read_model
 :reset_model!
 :set_callback_func!
 :set_objcoeffs!
 :set_sense!
 :setparam!
 :setparams!
 :tune_model
 :update_model!
 :upperbounds
 :write_model

In [4]:
##### Getting the License

# Env is a struct defined in the Gurobi module. Here we call one of its constructors.
# structs are documented here: https://docs.julialang.org/en/v1/manual/types/#Composite-Types
# constructors, here: https://docs.julialang.org/en/v1/manual/constructors/

# In documentation Env() is here: https://github.com/jump-dev/Gurobi.jl#reusing-the-same-gurobi-environment-for-multiple-solves
# In code here: https://github.com/jump-dev/Gurobi.jl/blob/master/src/MOI_wrapper/MOI_wrapper.jl#L92
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-09-16


Gurobi.Env(Ptr{Nothing} @0x0000000068fb92b0, false, 0)

In [5]:
##### Creating the Model

# Models in JuMP are documented here: https://jump.dev/JuMP.jl/stable/manual/models/
# The Model() constructor, here: https://jump.dev/JuMP.jl/stable/manual/models/#Create-a-model
# Or in code the struct is here: https://github.com/jump-dev/JuMP.jl/blob/master/src/JuMP.jl#L216
# and the constructor used is here: https://github.com/jump-dev/JuMP.jl/blob/master/src/JuMP.jl#L294
m = Model(() -> Gurobi.Optimizer(GRB_ENV))

# Question: What is () -> Gurobi.Optimizer(env)?? It's an anonymous function that takes in no inputs and
# returns an Optimizer struct. This function is passed from JuMP to MathOptInterface (which is a different
# module that JuMP relies on to convert algebraic models into a form that solvers can use) where it is actually
# called. Anonymous functions documented here: https://docs.julialang.org/en/v1/manual/functions/#man-anonymous-functions

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [6]:
##### DATAAAAAAA

# how I did this isn't really relevant so I'm not going to add many comments here

# generating the data
cath_x = Array{Float64}(undef,6^2)
cath_y = Array{Float64}(undef,6^2)
i=0
for x=1:6,y=1:6
    i=i+1 
    cath_x[i] = x
    cath_y[i] = y
end

loc_x = Array{Float64}(undef,6*9*9)
loc_y = Array{Float64}(undef,6*9*9)
loc_z = Array{Float64}(undef,6*9*9)
i = 0
for x=1:.5:5,y=1:.5:5,z=0:1:5
    i = i+1
    loc_x[i]=x
    loc_y[i]=y
    loc_z[i]=z
end

ncatheters = length(cath_x)
nlocs = length(loc_x)

d = [min( .5/((loc_x[i]-cath_x[j])^2+(loc_y[i]-cath_y[j])^2) , .5) for i=1:nlocs,j=1:ncatheters] # dose rate at location i from catheter j

# spherical tumor of radius 2 at x=y=z=3 that needs dose of at least 2 but not more than 5
# healthy tissue should receive dose less than 1
L = [sqrt( (loc_x[i]-3)^2+(loc_y[i]-3)^2+(loc_z[i]-3)^2 ) <= 2 ? 2 : 0 for i=1:nlocs]
U = [sqrt( (loc_x[i]-3)^2+(loc_y[i]-3)^2+(loc_z[i]-3)^2 ) <= 2 ? 5 : 1 for i=1:nlocs]

# especially sensitive cylindrical region centered at x=y=5 of radius 1
α = [1 for i=1:nlocs] # falling below minimum is penalized equally everywhere, however minimum for healthy tissue is 0 so will never be below minimum for healthy cells
β = [sqrt( (loc_x[i]-5)^2+(loc_y[i]-5)^2 ) <= 1 ? 5 : 1 for i=1:nlocs];  # exceeding max is worse in sensitive region and equally bad everywhere else

In [13]:
# saving all the data
using DataFrames, CSV, JLD2

cath_df = DataFrame(cath_x=cath_x,cath_y=cath_y)
loc_df = DataFrame(loc_x=loc_x,loc_y=loc_y,loc_z=loc_z)
bounds_df = DataFrame(L=L,U=U)
penalty_df = DataFrame(α=α,β=β)

CSV.write("cath_data.csv",cath_df)
CSV.write("loc_data.csv",loc_df)
CSV.write("bounds_data.csv",bounds_df)
CSV.write("penalty_data.csv",penalty_df)
JLD2.jldsave("dose_rate_data.jld2"; d)

In [8]:
# loading all the data
using DataFrames, CSV, JLD2

cath_df = CSV.read("cath_data.csv", DataFrame)
loc_df = CSV.read("loc_data.csv", DataFrame)
bounds_df = CSV.read("bounds_data.csv", DataFrame)
penalty_df = CSV.read("penalty_data.csv", DataFrame)

cath_x = cath_df[:,"cath_x"]
cath_y = cath_df[:,"cath_y"]

loc_x = loc_df[:,"loc_x"]
loc_y = loc_df[:,"loc_y"]
loc_z = loc_df[:,"loc_z"]

L = bounds_df[:,"L"]
U = bounds_df[:,"U"]

α = penalty_df[:,"α"]
β = penalty_df[:,"β"]

ncatheters = length(cath_x)
nlocs = length(loc_x)

d = load("dose_rate_data.jld2","d");

In [9]:
##### Defining the Problem

##### Defining variables: https://jump.dev/JuMP.jl/stable/manual/variables/

# Understanding JuMP's "containers" is important: https://jump.dev/JuMP.jl/stable/manual/containers/#Containers
# @variable is a "Macro" documented here: https://docs.julialang.org/en/v1/manual/metaprogramming/#man-macros
@variable(m,y[1:nlocs] >= 0) # documented in the JuMP manual here: https://jump.dev/JuMP.jl/stable/manual/variables/#Containers-of-variables

486-element Vector{VariableRef}:
 y[1]
 y[2]
 y[3]
 y[4]
 y[5]
 y[6]
 y[7]
 y[8]
 y[9]
 y[10]
 y[11]
 y[12]
 y[13]
 ⋮
 y[475]
 y[476]
 y[477]
 y[478]
 y[479]
 y[480]
 y[481]
 y[482]
 y[483]
 y[484]
 y[485]
 y[486]

In [10]:
Containers.@container([i=1:5],i)

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [11]:
Containers.@container([i=1:5,j=1:5],(i,j))

5×5 Matrix{Tuple{Int64, Int64}}:
 (1, 1)  (1, 2)  (1, 3)  (1, 4)  (1, 5)
 (2, 1)  (2, 2)  (2, 3)  (2, 4)  (2, 5)
 (3, 1)  (3, 2)  (3, 3)  (3, 4)  (3, 5)
 (4, 1)  (4, 2)  (4, 3)  (4, 4)  (4, 5)
 (5, 1)  (5, 2)  (5, 3)  (5, 4)  (5, 5)

In [12]:
Containers.@container([i=["hi","im"],j=["im","john"];i!=j], i * " magic " * j)

JuMP.Containers.SparseAxisArray{String, 2, Tuple{String, String}} with 3 entries:
  [im, john]  =  im magic john
  [hi, im  ]  =  hi magic im
  [hi, john]  =  hi magic john

In [13]:
@variable(m,t[1:ncatheters] >= 0);

In [14]:
@variable(m,z[1:ncatheters],Bin); # documented in the JuMP manual here: https://jump.dev/JuMP.jl/stable/manual/variables/#Binary-variables

In [15]:
##### Defining constraints: https://jump.dev/JuMP.jl/stable/manual/constraints/

In [16]:
@constraint(m,[i=1:nlocs],y[i] >= α[i]*(L[i]-sum(d[i,j]*t[j] for j=1:ncatheters)) ); # I'm creating a container of anonymous constraints: https://jump.dev/JuMP.jl/stable/manual/constraints/#Anonymous-constraints
                                                                                     # the "sum" is using julia's generator syntax: https://docs.julialang.org/en/v1/manual/arrays/#man-comprehensions
                                                                                     # and also documented here: https://docs.julialang.org/en/v1/manual/arrays/#Generator-Expressions

In [17]:
@constraint(m,[i=1:nlocs],y[i] >= β[i]*(sum(d[i,j]*t[j] for j=1:ncatheters)-U[i]) );

In [18]:
M = maximum(L)/minimum(d)       # Rule of thumb for faster solving: pick the smallest possible M you can.
                                # the largest possible t_j would be one catheter delivering the largest minimal dose to the slowest to reach region
@constraint(m,[j=1:ncatheters],t[j]<=M*z[j])

36-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 t[1] - 200 z[1] <= 0.0
 t[2] - 200 z[2] <= 0.0
 t[3] - 200 z[3] <= 0.0
 t[4] - 200 z[4] <= 0.0
 t[5] - 200 z[5] <= 0.0
 t[6] - 200 z[6] <= 0.0
 t[7] - 200 z[7] <= 0.0
 t[8] - 200 z[8] <= 0.0
 t[9] - 200 z[9] <= 0.0
 t[10] - 200 z[10] <= 0.0
 t[11] - 200 z[11] <= 0.0
 t[12] - 200 z[12] <= 0.0
 t[13] - 200 z[13] <= 0.0
 ⋮
 t[25] - 200 z[25] <= 0.0
 t[26] - 200 z[26] <= 0.0
 t[27] - 200 z[27] <= 0.0
 t[28] - 200 z[28] <= 0.0
 t[29] - 200 z[29] <= 0.0
 t[30] - 200 z[30] <= 0.0
 t[31] - 200 z[31] <= 0.0
 t[32] - 200 z[32] <= 0.0
 t[33] - 200 z[33] <= 0.0
 t[34] - 200 z[34] <= 0.0
 t[35] - 200 z[35] <= 0.0
 t[36] - 200 z[36] <= 0.0

In [19]:
N = 10
@constraint(m,sum(z) <= N)

z[1] + z[2] + z[3] + z[4] + z[5] + z[6] + z[7] + z[8] + z[9] + z[10] + z[11] + z[12] + z[13] + z[14] + z[15] + z[16] + z[17] + z[18] + z[19] + z[20] + z[21] + z[22] + z[23] + z[24] + z[25] + z[26] + z[27] + z[28] + z[29] + z[30] + z[31] + z[32] + z[33] + z[34] + z[35] + z[36] <= 10.0

In [20]:
# the neighbor constraint is trickier

# functions and methods are two different things in julia: https://docs.julialang.org/en/v1/manual/functions/ AND https://docs.julialang.org/en/v1/manual/methods/
function isneighbor(j,k,cath_x,cath_y)
    
    # basic boolean opertors: https://docs.julialang.org/en/v1/manual/mathematical-operations/#Boolean-Operators
    # numeric comparisons: https://docs.julialang.org/en/v1/manual/mathematical-operations/#Numeric-Comparisons
    
    sq_dist = (cath_x[j]-cath_x[k])^2 + (cath_y[j]-cath_y[k])^2
    
    if sq_dist >= 1 && sq_dist <= 2
        return true
    else
        return false
    end
end

@constraint(m,[j=1:ncatheters,k=1:ncatheters; j < k && isneighbor(j,k,cath_x,cath_y)], z[j]+z[k] <= 1) # note that the generator syntax and the JuMP container syntax is slightly different (; vs. if for the condition)

JuMP.Containers.SparseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 2, Tuple{Int64, Int64}} with 110 entries:
  [11, 17]  =  z[11] + z[17] <= 1.0
  [11, 18]  =  z[11] + z[18] <= 1.0
  [22, 28]  =  z[22] + z[28] <= 1.0
  [23, 24]  =  z[23] + z[24] <= 1.0
  [8, 15 ]  =  z[8] + z[15] <= 1.0
  [28, 34]  =  z[28] + z[34] <= 1.0
  [25, 26]  =  z[25] + z[26] <= 1.0
  [7, 8  ]  =  z[7] + z[8] <= 1.0
  [30, 35]  =  z[30] + z[35] <= 1.0
  [5, 11 ]  =  z[5] + z[11] <= 1.0
            ⋮
  [7, 14 ]  =  z[7] + z[14] <= 1.0
  [21, 28]  =  z[21] + z[28] <= 1.0
  [15, 21]  =  z[15] + z[21] <= 1.0
  [2, 3  ]  =  z[2] + z[3] <= 1.0
  [17, 23]  =  z[17] + z[23] <= 1.0
  [24, 29]  =  z[24] + z[29] <= 1.0
  [12, 17]  =  z[12] + z[17] <= 1.0
  [12, 18]  =  z[12] + z[18] <= 1.0
  [22, 23]  =  z[22] + z[23] <= 1.0
  [22, 29]  =  z[22] + z[29] <= 1.0
  [10, 16]  =  z[10] + z[16] <= 1.0

In [21]:
##### Defining the objective
@objective(m,Min,sum(y)); # objectives documented here: https://jump.dev/JuMP.jl/stable/manual/objective/

In [22]:
optimize!(m) # documented here: https://jump.dev/JuMP.jl/stable/reference/solutions/#Basic-utilities

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1119 rows, 558 columns and 36292 nonzeros
Model fingerprint: 0xb1f6eb15
Variable types: 522 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 250.0000000
Presolve removed 882 rows and 359 columns
Presolve time: 0.02s
Presolved: 237 rows, 199 columns, 6775 nonzeros
Variable types: 167 continuous, 32 integer (32 binary)

Root relaxation: objective 1.210000e+02, 300 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  121.00000    0   12  250.00000  121.00000  51.6%     -    0s
H    0     0                     191.4124209  12

In [23]:
t_opt = value.(t); # interacting with the solutions are documented here: https://jump.dev/JuMP.jl/stable/manual/solutions/

In [24]:
t_hm = reshape(round.(t_opt,digits=4),(6,6))

6×6 Matrix{Float64}:
 0.8018  0.0  0.0     0.6086  0.0     0.2517
 0.0     0.0  0.0     0.0     0.0     0.0
 0.0     0.0  3.2587  0.0     0.0     0.7256
 0.6086  0.0  0.0     0.0     0.0     0.0
 0.0     0.0  0.0     0.0     0.5836  0.0
 0.2517  0.0  0.7256  0.0     0.0     0.0

In [25]:
tumor = findall( i -> sqrt( (loc_x[i]-3)^2+(loc_y[i]-3)^2+(loc_z[i]-3)^2 ) <= 2, collect(1:nlocs)) # in tumor
# average radiation to tumor (2 to 5 target)
sum([sum(t_opt[j]*d[i,j] for j=1:ncatheters) for i in tumor])/length(tumor)

1.5033756118126462

In [26]:
sensitive = findall( i-> sqrt( (loc_x[i]-5)^2+(loc_y[i]-5)^2 ) <= 1 , collect(1:nlocs))
# average radiation to sensitive region (0 to 1 target)
sum([sum(t_opt[j]*d[i,j] for j=1:ncatheters) for i in sensitive])/length(sensitive)

0.841297425131274

In [27]:
not_tumor = findall( i -> sqrt( (loc_x[i]-3)^2+(loc_y[i]-3)^2+(loc_z[i]-3)^2 ) > 2, collect(1:nlocs)) # not in
# average radiation not to tumor
sum([sum(t_opt[j]*d[i,j] for j=1:ncatheters) for i in not_tumor])/length(not_tumor)

1.0746194108012872